In [31]:
import sys

# Add the parent directory to sys.path
sys.path.append("/home/jovyan/video-storage/amit_files/")

# Add the parent directory to sys.path
sys.path.append("/home/jovyan/video-storage/amit_files/image_background_remove_tool")


import torch
device = "cuda" if torch.cuda.is_available else "cpu"

In [4]:
!pip install gym

In [28]:
from carvekit.ml.arch.tracerb7.efficientnet import EfficientEncoderB7
from carvekit.ml.wrap.tracer_b7 import TracerUniversalB7

In [29]:
model = TracerUniversalB7()

In [32]:
model = model.to(device)

In [5]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import Adam
from torch.cuda.amp import autocast, GradScaler

from keypoints.keypoints.data_augments import TpsAndRotate, nop
from keypoints.keypoints.models import keynet
from keypoints.keypoints.utils import ResultsLogger
from keypoints.keypoints.ds import datasets as ds
from keypoints.keypoints.config import config

from keypoints.keypoints.ds.datasets import split

2024-11-25 20:04:36.930022: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-25 20:04:37.867643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
import yaml
import argparse
from pathlib import Path
import torch
import json
import numpy as np
from tqdm import tqdm

args_path = '/home/jovyan/video-storage/amit_files/keypoints/configs/keypoints_celeba.yaml'

In [7]:
torch.cuda.set_device(0)

In [12]:
args = config(['--config', args_path,'--data_root','/home/jovyan/video-storage/amit_files/Master_Thesis_Project/'])
print(args)

Namespace(device=device(type='cuda', index=0), run_id=3, comment=None, demo=False, load=None, transfer_load=None, checkpoint_freq=1000, data_root='/home/jovyan/video-storage/amit_files/Master_Thesis_Project/', config='/home/jovyan/video-storage/amit_files/keypoints/configs/keypoints_celeba.yaml', epochs=20, processes=7, seed=None, cma_algo=None, cma_step_mode='auto', cma_step_decay=0.001, cma_initial_step_size=None, cma_samples=None, cma_oversample=0, display=False, display_freq=5000, display_kp_rows=4, opt_level='O2', model_type='F', model_in_channels=3, model_keypoints=10, transporter_combine_mode='max', policy_action_select_mode='argmax', policy_depth=1, gym_reward_count_limit=None, optimizer='Adam', batch_size=16, lr=None, dataset='celeba', dataset_train_len=40000, dataset_test_len=1100, dataset_randomize=False, data_aug_tps_cntl_pts=4, data_aug_tps_variance=0.05, data_aug_max_rotate=0.1, model_z_channels=64, data_aug_type='tps_and_rotate', **{'cma_algo;': 'fast'})


In [9]:
run_dir = f'data/models/keypoints/{args.model_type}/run_{args.run_id}'
run_dir

'data/models/keypoints/F/run_2'

In [10]:
display = ResultsLogger(run_dir=run_dir,
                            num_keypoints=args.model_keypoints,
                            title = 'Results',
                            visuals = args.display,
                            image_capture_freq = args.display_freq,
                            kp_rows = args.display_kp_rows,
                            comment = args.comment)
display.header(args)

device: cuda:0 run_id: 2 comment: None demo: False load: None transfer_load: None checkpoint_freq: 1000 data_root: /home/jovyan/video-storage/amit_files/Master_Thesis_Project/celeba-low/ config: /home/jovyan/video-storage/amit_files/keypoints/configs/keypoints_celeba.yaml epochs: 20 processes: 7 seed: None cma_algo: None cma_step_mode: auto cma_step_decay: 0.001 cma_initial_step_size: None cma_samples: None cma_oversample: 0 display: False display_freq: 5000 display_kp_rows: 4 opt_level: O2 model_type: F model_in_channels: 3 model_keypoints: 10 transporter_combine_mode: max policy_action_select_mode: argmax policy_depth: 1 gym_reward_count_limit: None optimizer: Adam batch_size: 16 lr: None dataset: celeba dataset_train_len: 40000 dataset_test_len: 1100 dataset_randomize: False data_aug_tps_cntl_pts: 4 data_aug_tps_variance: 0.05 data_aug_max_rotate: 0.1 model_z_channels: 64 data_aug_type: tps_and_rotate cma_algo;: fast 


In [13]:
""" dataset """
datapack = ds.datasets[args.dataset]
train, test = datapack.make(args.dataset_train_len, args.dataset_test_len, data_root=args.data_root)
pin_memory = False if args.device == 'cpu' else True
train_l = DataLoader(train, batch_size=args.batch_size, shuffle=True, drop_last=True, pin_memory=pin_memory)
test_l = DataLoader(test, batch_size=args.batch_size, shuffle=True, drop_last=True, pin_memory=pin_memory)

In [17]:
""" model """
kp_network = keynet.make(args).to(args.device)

In [18]:
kp_network

KeyNet(
  (encoder): Unit(
    (in_block): Sequential(
      (0): ReplicationPad2d((1, 1, 1, 1))
      (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (core): Sequential(
      (0): ReplicationPad2d((1, 1, 1, 1))
      (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.01, inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): ReplicationPad2d((1, 1, 1, 1))
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
      (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): LeakyReLU(negative_slope=0.01, inplace=True)
      (9): ReplicationPad2d((1, 1, 1, 1))
      (10): Conv2d(256, 256, kern

In [19]:
optim = Adam(kp_network.parameters(), lr=1e-4)

In [20]:
""" data augmentation """
if args.data_aug_type == 'tps_and_rotate':
    augment = TpsAndRotate(args.data_aug_tps_cntl_pts, args.data_aug_tps_variance, args.data_aug_max_rotate)
else:
    augment = nop

In [22]:
import torch
import numpy as np

# loss function
def l2_reconstruction_loss(x, x_, threshold = 0.50, foreground_mask=None):
    # Calculate L2 loss
    loss = (x - x_) ** 2
    
    # Apply the foreground mask if provided
    if foreground_mask is not None:
        if not isinstance(foreground_mask, torch.Tensor):
            foreground_mask = torch.tensor(foreground_mask)
        # Use torch.where for PyTorch tensors
        loss = torch.where(foreground_mask >= threshold, loss * foreground_mask, loss * 0.5)
    
    # Return the mean loss
    return torch.mean(loss)


In [23]:
scaler = GradScaler()
threshold = 0.5

# loss function
# def l2_reconstruction_loss(x, x_, threshold = 0.50,foreground_mask=None):
#     loss = (x - x_) ** 2
#     if foreground_mask is not None:
#         loss = np.where(foreground_mask >= threshold, loss * foreground_mask, loss * 0.5)
#         # loss = loss * loss_mask
#     return torch.mean(loss)

criterion = l2_reconstruction_loss

def to_device(data, device):
    return tuple([x.to(device) for x in data])

In [24]:
# Initialize lists to store losses for each epoch
train_losses = []
val_losses = []

# Initialize the loss file before training
loss_file_path = run_dir + '/losses.json'
with open(loss_file_path, 'w') as f:
    json.dump({'train_losses': [], 'val_losses': []}, f)


In [50]:
from PIL import Image

# Main training loop
for epoch in tqdm(range(1, args.epochs + 1)):
    epoch_train_loss = 0
    epoch_val_loss = 0
    num_train_batches = 0
    num_val_batches = 0

    if not args.demo:
        # Training
        # batch = tqdm(trip_train, total=len(train_t) // args.batch_size)
        batch = tqdm(train_l, total=len(train) // args.batch_size)
        for i, data in enumerate(batch):
            data = to_device(data, device=args.device)
            # x, x_ = data[0].to(args.device), data[1].to(args.device)
            x, x_, loss_mask = augment(*data)
            # print(x_.shape)   
            # x__ = x_.detach().cpu().numpy()
            x__ = np.transpose(x_.detach().cpu().numpy(),(0, 2, 3, 1)).astype(np.uint8)
            # x__ = x__.astype(np.uint8)
            # print(x__.shape)
            x__ = [Image.fromarray(x__[i]) for i in range(x__.shape[0])]
            # print(type(x__))
            foreground_mask = model(x__)
            print(foreground_mask[0])
            
            optim.zero_grad()

            # Mixed precision forward pass
            with autocast():
                x_t, z, k, m, p, heatmap = kp_network(x, x_)
                loss = criterion(x_t, x_, threshold = threshold,foreground_mask = foreground_mask)
                epoch_train_loss += loss.item()
                num_train_batches += 1

            # Scaled backward pass
            scaler.scale(loss).backward()
            scaler.step(optim)    # Update model parameters
            scaler.update()       # Adjust the scale for next iteration

            if i % args.checkpoint_freq == 0:
                kp_network.save(run_dir + '/checkpoint')
                
            display.log(batch, epoch, i, loss, optim, x, x_, x_t, heatmap, k, m, p, loss_mask=None, type='train', depth=20)

    # Calculate average train loss for the epoch
    epoch_train_loss /= max(1, num_train_batches)
    train_losses.append(epoch_train_loss)

    # Validation
    with torch.no_grad():
        batch = tqdm(test_l, total=len(test) // args.batch_size)
        for i, data in enumerate(batch):
            data = augment(*data)
            x, x_, loss_mask = augment(*data)
            foreground_mask = model(x_)

            with autocast():
                x_t, z, k, m, p, heatmap = kp_network(x, x_)
                loss = criterion(x_t, x_, threshold = threshold,foreground_mask = foreground_mask)
                epoch_val_loss += loss.item()
                num_val_batches += 1

            display.log(batch, epoch, i, loss, optim, x, x_, x_t, heatmap, k, m, p, loss_mask=None, type='test', depth=20)

    # Calculate average validation loss for the epoch
    epoch_val_loss /= max(1, num_val_batches)
    val_losses.append(epoch_val_loss)

    # Save the updated train and val losses after each epoch
    with open(loss_file_path, 'r+') as f:
        loss_data = json.load(f)
        loss_data['train_losses'].append(epoch_train_loss)
        loss_data['val_losses'].append(epoch_val_loss)
        f.seek(0)  # Go to the beginning of the file
        json.dump(loss_data, f, indent=4)  # Write updated data
        f.truncate()  # Remove any leftover data from previous writes

    ave_loss, best_loss = display.end_epoch(epoch, optim)

    # Save if model improved
    if ave_loss <= best_loss and not args.demo:
        kp_network.save(run_dir + '/best')


  0%|                                                                                  | 0/20 [00:03<?, ?it/s]

<PIL.Image.Image image mode=L size=128x128 at 0x7FB1F03BCA90>


RuntimeError: Could not infer dtype of Image